In [2]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 1.4MB/s 
     |████████████████████████████████| 6.8MB 44.2MB/s 
     |████████████████████████████████| 983kB 41.0MB/s 
     |████████████████████████████████| 266kB 46.6MB/s 
     |████████████████████████████████| 1.3MB 41.5MB/s 
     |████████████████████████████████| 1.1MB 47.4MB/s 
     |████████████████████████████████| 471kB 48.0MB/s 
     |████████████████████████████████| 890kB 25.9MB/s 
     |████████████████████████████████| 2.9MB 47.7MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.3-cp36-none-any.whl size=25276312 sha256=9b3783d5a85ece05e41970ab4c80e1829d9463d1b2621d81c4223db13d940af2
  Stored in directory: /root/.cache/pip/wheels/86/cd/9c/ad53b98a3eccf4ddeddccfce99d353781f206a1ac4287ca721
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993194 sha256=814d17d5359aef8506231f3813acc8c618a24382e3e8b45fa536b2e5cfe8db28
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

In [3]:
import tensorflow as tf

In [4]:
tf.__version__

'2.4.0'

In [ ]:
#!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

In [5]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf

In [6]:
data_train = pd.read_excel('/content/drive/MyDrive/Data/mytrain.xlsx', dtype = str)

In [7]:
data_test = pd.read_excel('/content/drive/MyDrive/Data/mytest.xlsx', dtype = str)

In [8]:
data_train.tail()

,Reviews,Sentiment
239227,your standard Hollywood bio-pic,disappointed
239228,your typical ` fish out of water ' story,disappointed
239229,zero .,disappointed
239230,zippy jazzy score,delighted
239231,UNK,neutral


In [9]:
data_test.tail()

,Reviews,Sentiment
15568,you're in no way i asked about a product that...,happy
15569,you're in no way i just got the service i wan...,happy
15570,you're ok,neutral
15571,zoe was very good listened to my problem and ...,happy
15572,zoi was really helpful,happy


In [10]:
data_test.shape, data_train.shape

((15573, 2), (239232, 2))

In [19]:
(X_train, y_train), (X_test, y_test), preproc = text.texts_from_df(train_df=data_train,
                                                                   text_column = 'Reviews',
                                                                   label_columns = 'Sentiment',
                                                                   val_df = data_test,
                                                                   maxlen = 500,
                                                                   preprocess_mode = 'bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [20]:
model = text.text_classifier(name = 'bert',
                             train_data = (X_train, y_train),
                             preproc = preproc)

Is Multi-Label? False
maxlen is 500
done.


In [21]:
learner = ktrain.get_learner(model=model, train_data=(X_train, y_train),
                   val_data = (X_test, y_test),
                   batch_size = 6)

In [ ]:
learner.fit_onecycle(lr = 2e-5, epochs = 1)



begin training using onecycle policy with max lr of 2e-05...
39872/39872 [==============================] - 14213s 356ms/step - loss: 1.0158 - accuracy: 0.5631 - val_loss: 1.8129 - val_accuracy: 0.3772


In [22]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [25]:
data = ['completely waste of time',
        'very satisfied! thanx',
        'highly skilled person, could not be better',
        'faster service! still waiting for a cheque to be issued! use royal mail tracking instead of just blaming them. better staff that actually deal with the issue instead of just parming you off! better wording of emails',
        'great help ',
        'i was given a replacement toothbrush handle as mine had stopped working properly so that is why i am happy', 
        'very bad manners agent was rude shouted at me']

In [27]:
predictor.predict(data)

['irritated',
 'irritated',
 'irritated',
 'irritated',
 'irritated',
 'irritated',
 'irritated']

In [ ]:
predictor.predict(data, return_proba=True)

array([[0.08703343, 0.3069396 , 0.18387152, 0.28305125, 0.13910414],
       [0.09324206, 0.27936772, 0.21819992, 0.2772791 , 0.13191114],
       [0.06329511, 0.4051564 , 0.20668468, 0.19334635, 0.1315174 ],
       [0.09769331, 0.4151423 , 0.13870363, 0.2231487 , 0.12531197],
       [0.08713117, 0.21421917, 0.19776125, 0.35557404, 0.14531441],
       [0.07745183, 0.44616476, 0.10952632, 0.26049057, 0.10636648],
       [0.08854856, 0.23882023, 0.17602955, 0.36229923, 0.13430244]],
      dtype=float32)

In [ ]:
predictor.get_classes()

['delighted', 'disappointed', 'happy', 'irritated', 'neutral']

In [ ]:
#predictor.save('/content/drive/MyDrive/Data')

In [ ]:
#!zip -r /content/bert.zip /content/drive/MyDrive/Data

  adding: content/drive/MyDrive/Data/ (stored 0%)
  adding: content/drive/MyDrive/Data/tf_model.preproc (deflated 52%)
  adding: content/drive/MyDrive/Data/tf_model.h5 (deflated 16%)
  adding: content/drive/MyDrive/Data/mytest.xlsx (deflated 2%)
  adding: content/drive/MyDrive/Data/mytrain.xlsx (deflated 9%)
  adding: content/drive/MyDrive/Data/.ipynb_checkpoints/ (stored 0%)


In [31]:
predictor_load = ktrain.load_predictor('/content/drive/MyDrive/Data')

In [32]:
data = ['completely waste of time',
        'very satisfied! thanx',
        'highly skilled person, could not be better',
        'faster service! still waiting for a cheque to be issued! use royal mail tracking instead of just blaming them. better staff that actually deal with the issue instead of just parming you off! better wording of emails',
        'great help ',
        'i was given a replacement toothbrush handle as mine had stopped working properly so that is why i am happy', 
        'very bad manners agent was rude shouted at me']

In [33]:
predictor_load.get_classes()

['delighted', 'disappointed', 'happy', 'irritated', 'neutral']

In [34]:
predictor_load.predict(data)

['irritated',
 'happy',
 'delighted',
 'disappointed',
 'happy',
 'disappointed',
 'irritated']

In [35]:
predictor_load.predict(data, return_proba=True)

array([[1.84147284e-04, 1.94262743e-01, 3.88112478e-03, 7.99516201e-01,
        2.15577637e-03],
       [3.52644354e-01, 2.51424760e-02, 6.01704240e-01, 1.03789556e-03,
        1.94710437e-02],
       [5.85029840e-01, 1.84306931e-02, 3.90015662e-01, 1.03385933e-03,
        5.49004227e-03],
       [6.68130210e-03, 6.17596626e-01, 1.87935472e-01, 1.58298790e-01,
        2.94878073e-02],
       [4.26579028e-01, 4.51101037e-03, 5.65010786e-01, 1.20634977e-04,
        3.77854658e-03],
       [4.91284914e-02, 4.37433422e-01, 4.06600475e-01, 7.49518350e-02,
        3.18858251e-02],
       [2.66982184e-04, 4.58632737e-01, 1.54394545e-02, 5.16658485e-01,
        9.00238194e-03]], dtype=float32)